In [16]:
from tqdm import tqdm
from collections import Counter, defaultdict
import random
from copy import copy, deepcopy
import numpy as np

from dataclasses import dataclass, field
import time

In [3]:
# %%bash
# FILE_NAME="dijkstraData.txt"
# LINK="https://d3c33hcgiwev3.cloudfront.net/_dcf1d02570e57d23ab526b1e33ba6f12_dijkstraData.txt?Expires=1618272000&Signature=Tiq7JQlTIpFGpeLb8oqcZx-Ynpw~mDXBlxWDxpyI91vQUFyhErkiJG9DLaNnRJcCgK8hBcX94KFkGvLMAE76TDF9g9v6e~GolRXA527rscMZU~VdycvLO5H6oOtptnIF28zOs10sJxHmRokqDDhmV2TSKlnNSQZC10WmqaJnLt0_&Key-Pair-Id=APKAJLTNE6QMUY6HBC5A"

# wget -O $FILE_NAME $LINK 

In [6]:
FILE_NAME="dijkstraData.txt"

with open(FILE_NAME) as file:
    a = file.read().splitlines()

In [7]:
a[:5]

['1\t80,982\t163,8164\t170,2620\t145,648\t200,8021\t173,2069\t92,647\t26,4122\t140,546\t11,1913\t160,6461\t27,7905\t40,9047\t150,2183\t61,9146\t159,7420\t198,1724\t114,508\t104,6647\t30,4612\t99,2367\t138,7896\t169,8700\t49,2437\t125,2909\t117,2597\t55,6399\t',
 '2\t42,1689\t127,9365\t5,8026\t170,9342\t131,7005\t172,1438\t34,315\t30,2455\t26,2328\t6,8847\t11,1873\t17,5409\t157,8643\t159,1397\t142,7731\t182,7908\t93,8177\t',
 '3\t57,1239\t101,3381\t43,7313\t41,7212\t91,2483\t31,3031\t167,3877\t106,6521\t76,7729\t122,9640\t144,285\t44,2165\t6,9006\t177,7097\t119,7711\t',
 '4\t162,3924\t70,5285\t195,2490\t72,6508\t126,2625\t121,7639\t31,399\t118,3626\t90,9446\t127,6808\t135,7582\t159,6133\t106,4769\t52,9267\t190,7536\t78,8058\t75,7044\t116,6771\t49,619\t107,4383\t89,6363\t54,313\t',
 '5\t200,4009\t112,1522\t25,3496\t23,9432\t64,7836\t56,8262\t120,1862\t2,8026\t90,8919\t142,1195\t81,2469\t182,8806\t17,2514\t83,8407\t146,5308\t147,1087\t51,22\t']

# Dijkstra

## With heap

In [8]:
import heapq

@dataclass
class Edge:
    src: int
    dst: int
    weight: int
        
@dataclass
class Graph:
    V: set
    v_to_edgelist: dict

In [9]:
V = set()
v_to_edgelist = {}
for x in a:
    x = x.split('\t')
    src = int(x[0])
    egdes = []
    for y in x[1:-1]:
        y = y.split(',')
        egdes.append( Edge(src=src, dst=int(y[0]), weight=int(y[1])) )
    V.add(src)
    v_to_edgelist[src] = egdes

In [29]:
def Dijkstra_SSSP(G, start_v):
    G = deepcopy(G)
    D = defaultdict(lambda: np.inf)
    curr_v = start_v
    G.V.remove(curr_v)
    D[curr_v] = 0
    
    heap = []
    while G.V:
        if curr_v in G.v_to_edgelist:
            edges = G.v_to_edgelist[curr_v]
            for edge in edges:
                new_dist = D[edge.src] + edge.weight
                if new_dist < D[edge.dst]:
                    D[edge.dst] = new_dist
                    heapq.heappush(heap, (new_dist, edge.dst))
        
        dist, dst = heapq.heappop(heap)
        while dst not in G.V: # clean heap from edges whos v are already in D 
            dist, dst = heapq.heappop(heap) # it will stop when finds dst that is in G.V (not explored)
        G.V.remove(dst)
        curr_v = dst
    D = dict(D)
    return D

In [30]:
%%time
G = Graph(V, v_to_edgelist)
D = Dijkstra_SSSP(G, 1)
targets = [7, 37, 59, 82, 99, 115, 133, 165, 188, 197]
','.join([str(D[t]) for t in targets])

CPU times: user 61.6 ms, sys: 2.1 ms, total: 63.7 ms
Wall time: 62.4 ms


'2599,2610,2947,2052,2367,2399,2029,2442,2505,3068'

## Greedy

In [ ]:
D = dict()
for x in a:
    x = x.split('\t')
    D[int(x[0])] = [tuple(map(int, y.split(','))) for y in x[1:] if y != '']

In [22]:
def dijkstra(D):
    A = defaultdict(lambda:1000000)
    X = set([1])
    V = set(D.keys())
    A[1] = 0
    
    while len(X) != len(V):
        temp = []
        for v in X:
            for edge in D[v]:
                to_v = edge[0]
                if to_v not in X:
                    length = edge[1]
                    score = A[v] + length
                    temp.append((to_v, score))
        to_v, score = min(temp, key=lambda x: x[1])  
        A[to_v] = score
        X.add(to_v)
    return A

In [24]:
A = dijkstra(D)
targets = [7, 37, 59, 82, 99, 115, 133, 165, 188, 197]
[A[t] for t in targets]

[2599, 2610, 2947, 2052, 2367, 2399, 2029, 2442, 2505, 3068]

In [ ]:
# [2599, 2610, 2947, 2052, 2367, 2399, 2029, 2442, 2505, 3068]